### **Célula 1: Setup do Ambiente e Carregamento de Dados**

Esta célula é responsável por toda a preparação inicial do nosso ambiente de execução. As ações realizadas são:
1.  **Instalação de Bibliotecas:** Instala as dependências necessárias (`openai`, `pandas`, `openpyxl`) de forma silenciosa.
2.  **Importação de Módulos:** Importa todas as bibliotecas que serão utilizadas ao longo do notebook, como `pandas` para manipulação de dados e `OpenAI` para a comunicação com o agente de IA.
3.  **Configuração do Logger:** Configura um sistema de logs profissional para que possamos acompanhar cada passo da execução do agente, facilitando a identificação de problemas.
4.  **Conexão Segura com a API:** Estabelece a conexão com a API da OpenRouter. Ela lê a chave de API de forma segura a partir do gerenciador de "Secrets" do Colab, evitando a exposição de informações sensíveis diretamente no código.
5.  **Carregamento de Dados Brutos:** Carrega todos os arquivos Excel de entrada para a memória, armazenando cada um em um `DataFrame` do `pandas` dentro de um dicionário chamado `dataframes_brutos`.

In [183]:
# Célula 1: Setup Completo do Ambiente e Carregamento de Dados

#   Instala as bibliotecas necessárias
!pip install -q -U openai pandas openpyxl

#   Importa os módulos essenciais
import pandas as pd
import numpy as np
from pathlib import Path
from dataclasses import dataclass, field
from datetime import datetime, timedelta, date
import calendar
import re
import logging
import json
from IPython.display import display
from google.colab import userdata
from openai import OpenAI

#  Configuração robusta do Logger
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(name)s] - [%(levelname)s] - %(message)s")
logger = logging.getLogger("FluxoPrincipal")

#  Carregamento seguro da Chave de API e configuração do Cliente de IA
try:
    OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KE')
    client_ia = OpenAI(api_key=OPENROUTER_API_KEY, base_url="https://openrouter.ai/api/v1")
    logger.info("✅ Conexão com a API da OpenRouter foi bem-sucedida.")
except Exception as e:
    logger.error(f"❌ ERRO ao configurar a API: {e}. Verifique se o secret 'OPENROUTER_API_KEY' está configurado.")
    client_ia = None

#  Carregamento dos arquivos de dados brutos
logger.info("--- ETAPA 1: Carregando todos os arquivos brutos para a memória ---")
arquivos_para_carregar = {
    "admissao": "ADMISSÃO ABRIL.xlsx", "afastamentos": "AFASTAMENTOS.xlsx", "aprendiz": "APRENDIZ.xlsx",
    "ativos": "ATIVOS.xlsx", "sindicatos": "Base sindicato x valor.xlsx", "desligados": "DESLIGADOS.xlsx",
    "estagio": "ESTÁGIO.xlsx", "exterior": "EXTERIOR.xlsx", "ferias": "FÉRIAS.xlsx"
}
dataframes_brutos = {nome: pd.read_excel(arquivo) for nome, arquivo in arquivos_para_carregar.items()}
logger.info(f"✅ {len(dataframes_brutos)} arquivos carregados com sucesso.")

2025-08-26 02:16:49,083 [FluxoPrincipal] - [INFO] - ✅ Conexão com a API da OpenRouter foi bem-sucedida.
2025-08-26 02:16:49,084 [FluxoPrincipal] - [INFO] - --- ETAPA 1: Carregando todos os arquivos brutos para a memória ---
2025-08-26 02:16:51,432 [FluxoPrincipal] - [INFO] - ✅ 9 arquivos carregados com sucesso.


### **Célula 2: Definição da Arquitetura do Agente de Cálculo (VRAgent)**

Esta é a célula mais importante do projeto, onde toda a inteligência e as regras de negócio são encapsuladas. Ela define a classe `VRAgent`, que funciona como nosso "especialista" em cálculo de VR.

A classe é estruturada da seguinte forma:

* **`__init__` e `AgentConfig`**: O construtor recebe um objeto de configuração com os parâmetros da missão (ano, mês, feriados, etc.), tornando o agente reutilizável para diferentes cenários.
* **`_padronizar`**: Primeira etapa do tratamento. Padroniza os nomes de todas as colunas dos `DataFrames` para minúsculas e sem espaços, facilitando a manipulação dos dados.
* **`_apply_exclusions`**: Aplica as regras de negócio para determinar quem é elegível ao benefício, excluindo funcionários com base em cargo (diretores, estagiários), status (férias, desligados) e outras condições.
* **`_get_valores_diarios_em_lote` (Função Otimizada):** Esta é a correção crucial. O agente identifica todas as combinações únicas de `estado` e `cargo` dos funcionários elegíveis e envia uma **única consulta consolidada** para a IA. Isso resolve o erro "429 Too Many Requests" e aumenta drasticamente a eficiência.
* **`_enrich_and_calculate` (Função Otimizada):** O coração do cálculo. Após receber a resposta da IA, esta função:
    1.  **Aplica os valores individualmente:** Usa a técnica de `merge` do `pandas` para atribuir o `valor_diario` correto a cada funcionário, com base na sua combinação de estado e cargo.
    2.  **Calcula os dias pagáveis:** Utiliza um motor de cálculo que considera admissão, demissão, feriados e férias.
    3.  **Calcula os valores finais:** Calcula o `total_vr`, `custo_empresa` e `desconto_colaborador`.
* **`run`**: O método principal que orquestra toda a execução, chamando os métodos privados na ordem correta e gerando o arquivo Excel final.

In [187]:
# Célula 2: Definição da Arquitetura do Agente (Corrigido)

# --- Funções Utilitárias e Classes de Configuração ---
def contains_any(text, patterns):
    if pd.isna(text): return False
    s = str(text).lower(); return any(p.lower() in s for p in patterns)

@dataclass
class AgentConfig:
    year: int; month: int; holidays: list = field(default_factory=list)
    exclude_roles_patterns: list = field(default_factory=lambda: ["diretor", "estagiario", "estágio", "aprendiz"])

# --- Classe Principal do Agente ---
class VRAgent:
    def __init__(self, config: AgentConfig):
        self.config = config
        self.report = {"warnings": [], "errors": [], "fixes": []}
        self.logger = logging.getLogger("VRAgent")
        self.logger.info("Agente de Cálculo instanciado.")

    def _get_valores_diarios_em_lote(self, combos_df: pd.DataFrame):
        """Recebe um DataFrame de combinações (estado, cargo) e retorna um DataFrame com o valor_diario para cada um."""
        if combos_df.empty:
            return pd.DataFrame(columns=['estado', 'cargo', 'valor_diario'])

        combos_str = combos_df.to_string(index=False)

        # O prompt foi ajustado para ser mais genérico e não usar uma variável 'estado' individual.
        prompt = f"""
Para cada combinação de estado e cargo listada abaixo, forneça o valor diário mínimo do vale-refeição, em um valor único, segundo a convenção coletiva mais recente para o setor de tecnologia no Brasil para o ano de {self.config.year}.

Lista de Combinações:
{combos_str}

Responda em formato de um objeto JSON contendo uma única chave "valores", que é uma lista de objetos. Cada objeto deve ter as chaves "estado", "cargo" e "valor_diario". Use ponto como separador decimal para o valor. Se não souber, retorne 0.0.

Exemplo de formato da resposta:
{{
  "valores": [
    {{"estado": "sao paulo", "cargo": "analista de sistemas", "valor_diario": 35.50}},
    {{"estado": "rio de janeiro", "cargo": "programador junior", "valor_diario": 32.00}}
  ]
}}
"""
        try:
            self.logger.info(f"Consultando IA para {len(combos_df)} combinações de estado/cargo de uma só vez... (Isso pode levar um momento)")
            response = client_ia.chat.completions.create(
                model="mistralai/mistral-7b-instruct:free",
                response_format={"type": "json_object"},
                messages=[{"role": "user", "content": prompt}]
            )
            content = response.choices[0].message.content
            data = json.loads(content)

            valores_df = pd.DataFrame(data.get("valores", []))
            self.logger.info(f"✅ Valores diários obtidos via IA para {len(valores_df)} combinações.")
            return valores_df

        except Exception as e:
            self.logger.error(f"❌ Erro crítico ao consultar IA para valores em lote: {e}")
            return pd.DataFrame(columns=['estado', 'cargo', 'valor_diario'])

    def _padronizar(self, dataframes: dict):
        dfs = {k: v.copy() for k, v in dataframes.items()}
        for nome, df in dfs.items():
            df.columns = [str(c).strip().lower() for c in df.columns]
            rename_map = {
                'matricula': 'cadastro', 'titulo do cargo': 'cargo', 'admissão': 'data_admissao',
                'data demissão': 'data_demissao', 'dias de férias': 'dias_ferias',
                'estado': 'sindicato_lookup', 'valor': 'valor_diario_base',
                'comunicado de desligamento': 'comunicado_desligamento'
            }
            df.rename(columns=lambda c: rename_map.get(c, c), inplace=True)
            if 'cadastro' in df.columns:
                df['cadastro'] = pd.to_numeric(df['cadastro'], errors='coerce')
        return dfs

    def _apply_exclusions(self, dfs: dict):
        base_unica = pd.concat([dfs['ativos'], dfs.get('admissao', pd.DataFrame())], ignore_index=True).drop_duplicates(subset=['cadastro'], keep='last').dropna(subset=['cadastro'])
        base_unica['cadastro'] = base_unica['cadastro'].astype(int)

        self.logger.info(f"Iniciando com {len(base_unica)} registros únicos.")
        ativa_mask = pd.Series(True, index=base_unica.index)

        if 'cargo' in base_unica.columns:
            mask = base_unica['cargo'].fillna("").apply(lambda s: contains_any(s, self.config.exclude_roles_patterns))
            if mask.any(): self.report["fixes"].append(f"Excluídos por cargo: {mask.sum()} registros")
            ativa_mask &= ~mask

        for k in ["ferias", "desligados", "exterior", "afastamentos", "estagio", "aprendiz"]:
            dfk = dfs.get(k)
            if dfk is not None and "cadastro" in dfk.columns:
                cads_para_excluir = set(dfk["cadastro"].dropna())
                mask_exclusao = base_unica["cadastro"].isin(cads_para_excluir)
                if mask_exclusao.sum() > 0: self.report["fixes"].append(f"Exclusão por '{k}': {mask_exclusao.sum()} cadastros")
                ativa_mask &= ~mask_exclusao

        df_validos = base_unica[ativa_mask].copy()
        self.logger.info(f"Base de ativos válidos após exclusões: {len(df_validos)} registros.")
        return df_validos

    def _enrich_and_calculate(self, df_validos, dfs):
        df = df_validos.copy()
        if df.empty:
            self.logger.warning("Nenhum funcionário válido restou após as exclusões. O arquivo final ficará vazio.")
            return df

        self.logger.info("Enriquecendo dados com informações de férias e desligamentos...")
        if 'ferias' in dfs and 'cadastro' in dfs['ferias'].columns:
            df = pd.merge(df, dfs['ferias'][['cadastro', 'dias_ferias']], on='cadastro', how='left')
        if 'desligados' in dfs and 'cadastro' in dfs['desligados'].columns:
            df = pd.merge(df, dfs['desligados'], on='cadastro', how='left')

        for col in ['data_admissao', 'data_demissao']:
            if col in df.columns: df[col] = pd.to_datetime(df[col], errors='coerce')

        mapa_estados = {'SP': 'sao paulo', 'RS': 'rio grande do sul', 'PR': 'parana', 'RJ': 'rio de janeiro'}
        def get_estado(s):
            if pd.isna(s): return None
            s_norm = str(s).upper()
            for sigla, nome in mapa_estados.items():
                if sigla in s_norm: return nome
            return None

        df['estado'] = df['sindicato'].apply(get_estado)
        df['cargo_norm'] = df['cargo'].fillna('não especificado').str.lower()

        combos_unicos_df = df[df['estado'].notna()][['estado', 'cargo_norm']].drop_duplicates().copy()
        combos_unicos_df.rename(columns={'cargo_norm': 'cargo'}, inplace=True)

        if not combos_unicos_df.empty:
            valores_diarios_ia_df = self._get_valores_diarios_em_lote(combos_unicos_df)

            if not valores_diarios_ia_df.empty:
                valores_diarios_ia_df['cargo'] = valores_diarios_ia_df['cargo'].str.lower()
                df = pd.merge(df, valores_diarios_ia_df, left_on=['estado', 'cargo_norm'], right_on=['estado', 'cargo'], how='left')
                if 'cargo_y' in df.columns: df.drop(columns=['cargo_y'], inplace=True)
                if 'cargo_x' in df.columns: df.rename(columns={'cargo_x': 'cargo'}, inplace=True)
            else:
                df['valor_diario'] = 0.0
        else:
            df['valor_diario'] = 0.0

        self.logger.info("Forçando a coluna 'valor_diario' para tipo numérico para evitar erros de cálculo.")
        df['valor_diario'] = pd.to_numeric(df['valor_diario'], errors='coerce').fillna(0.0)

        self.logger.info("Iniciando o motor de cálculo para dias pagáveis...")
        def motor_de_calculo(row, ano, mes, holidays):
            try:
                inicio_mes = date(ano, mes, 1)
                fim_mes = date(ano, mes, calendar.monthrange(ano, mes)[1])
                inicio_periodo = row['data_admissao'].date() if pd.notna(row.get('data_admissao')) and row['data_admissao'].date() > inicio_mes else inicio_mes
                fim_periodo = row['data_demissao'].date() if pd.notna(row.get('data_demissao')) and row['data_demissao'].date() < fim_mes else fim_mes
                if 'comunicado_desligamento' in row and row.get('comunicado_desligamento') == 'OK' and pd.notna(row.get('data_demissao')) and row['data_demissao'].date().day <= 15: return 0
                if inicio_periodo > fim_periodo: return 0
                dias_uteis = np.busday_count(inicio_periodo.strftime('%Y-%m-%d'), (fim_periodo + timedelta(days=1)).strftime('%Y-%m-%d'), holidays=[d.strftime("%Y-%m-%d") for d in holidays])
                dias_ferias = row.get('dias_ferias', 0); dias_ferias = dias_ferias if pd.notna(dias_ferias) else 0
                return max(0, dias_uteis - dias_ferias)
            except Exception:
                return 0

        df['dias_pagaveis'] = df.apply(lambda r: motor_de_calculo(r, self.config.year, self.config.month, self.config.holidays), axis=1)

        self.logger.info("Calculando valores finais de VR...")
        df['total_vr'] = df['dias_pagaveis'] * df['valor_diario']
        df['custo_empresa'] = df['total_vr'] * 0.8
        df['desconto_colaborador'] = df['total_vr'] * 0.2
        self.logger.info("Cálculos de dias e valores finalizados.")
        return df

    def run(self, dataframes_brutos: dict, output_path: str):
        self.logger.info("Iniciando missão: padronizar, excluir, enriquecer e calcular.")
        dfs = self._padronizar(dataframes_brutos)
        df_validos = self._apply_exclusions(dfs)
        df_calculado = self._enrich_and_calculate(df_validos, dfs)
        if df_calculado.empty or 'total_vr' not in df_calculado.columns:
             self.logger.warning("O DataFrame calculado está vazio ou incompleto. O arquivo de saída não conterá dados.")
             df_saida = pd.DataFrame(columns=["cadastro", "nome", "cargo", "sindicato", "dias_pagaveis", "valor_diario", "total_vr", "custo_empresa", "desconto_colaborador"])
        else:
            df_saida = df_calculado[df_calculado['total_vr'] > 0].copy()
            self.logger.info(f"{len(df_calculado) - len(df_saida)} registros com valor zero foram removidos da saída final.")

        colunas_finais = ["cadastro", "nome", "cargo", "sindicato", "dias_pagaveis", "valor_diario", "total_vr", "custo_empresa", "desconto_colaborador"]
        for col in colunas_finais:
            if col not in df_saida.columns: df_saida[col] = ''
        df_saida = df_saida[colunas_finais]
        df_saida.to_excel(output_path, index=False, sheet_name="VR_MENSAL")
        self.logger.info(f"Arquivo final salvo em: {output_path}")
        return df_saida, self.report

### **Célula 3: Configuração e Execução da Missão**

Esta é a célula de controle do nosso notebook. Aqui, nós configuramos e iniciamos o processo do agente.

1.  **Configuração da Missão (`AgentConfig`):** Definimos os parâmetros específicos para esta execução, como o `ano` e o `mês` do cálculo e a lista de `feriados` nacionais a serem considerados.
2.  **Criação e Execução do Agente:** Instanciamos a classe `VRAgent` com a configuração definida e chamamos o método `.run()`, que dispara todo o fluxo de processamento encapsulado na Célula 2.
3.  **Exibição dos Resultados:** Após a conclusão da missão, esta célula exibe:
    * Uma **amostra** (`.head()`) do `DataFrame` final, permitindo uma rápida verificação visual dos dados gerados.
    * Um **relatório de validação** que resume as ações importantes que o agente tomou, como o número de funcionários excluídos por cada regra.

In [188]:
# Célula 3: Configuração e Execução da Missão do Agente

# Configurar a missão de cálculo
config = AgentConfig(
    year=2025,
    month=5,
    holidays=[date(2025, 5, 1)]
)

# Criar e executar o Agente de Cálculo
agent = VRAgent(config)
df_saida, report = agent.run(dataframes_brutos, output_path="VR_PROCESSADO_FINAL.xlsx")

# Exibir os resultados
print("\n" + "="*50)
print(" MISSÃO DO AGENTE DE CÁLCULO CONCLUÍDA")
print("="*50)

print("\nAmostra da Planilha Final Gerada:")
# Garante que o display não falhe se o df_saida for None
if df_saida is not None:
    display(df_saida.head())
else:
    print("O DataFrame de saída está vazio ou nulo.")

print("\nRelatório de Validação e Ações do Agente:")
from pprint import pprint
pprint(report)

2025-08-26 02:22:21,759 [VRAgent] - [INFO] - Agente de Cálculo instanciado.
2025-08-26 02:22:21,761 [VRAgent] - [INFO] - Iniciando missão: padronizar, excluir, enriquecer e calcular.
2025-08-26 02:22:21,776 [VRAgent] - [INFO] - Iniciando com 1825 registros únicos.
2025-08-26 02:22:21,790 [VRAgent] - [INFO] - Base de ativos válidos após exclusões: 1722 registros.
2025-08-26 02:22:21,793 [VRAgent] - [INFO] - Enriquecendo dados com informações de férias e desligamentos...
2025-08-26 02:22:21,819 [VRAgent] - [INFO] - Consultando IA para 283 combinações de estado/cargo de uma só vez... (Isso pode levar um momento)
2025-08-26 02:22:22,181 [httpx] - [INFO] - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-26 02:24:24,427 [VRAgent] - [INFO] - ✅ Valores diários obtidos via IA para 247 combinações.
2025-08-26 02:24:24,439 [VRAgent] - [INFO] - Forçando a coluna 'valor_diario' para tipo numérico para evitar erros de cálculo.
2025-08-26 02:24:24,441 [VRAge


 MISSÃO DO AGENTE DE CÁLCULO CONCLUÍDA

Amostra da Planilha Final Gerada:


,cadastro,nome,cargo,sindicato,dias_pagaveis,valor_diario,total_vr,custo_empresa,desconto_colaborador
882,31153,,ASSISTENTE DE ADMINISTRACAO DE PESSOAS,SINDPD SP - SIND.TRAB.EM PROC DADOS E EMPR.EMP...,21,35.0,735.0,588.0,147.0
883,32795,,ANALISTA DE COMUNICACAO E MARKETING III,SINDPD SP - SIND.TRAB.EM PROC DADOS E EMPR.EMP...,21,60.0,1260.0,1008.0,252.0
884,30770,,DESENVOLVEDOR III,SINDPD SP - SIND.TRAB.EM PROC DADOS E EMPR.EMP...,21,65.0,1365.0,1092.0,273.0
885,30770,,DESENVOLVEDOR III,SINDPD SP - SIND.TRAB.EM PROC DADOS E EMPR.EMP...,21,75.0,1575.0,1260.0,315.0
886,34118,,CONSULTOR FUNCIONAL SAP III,SINDPD SP - SIND.TRAB.EM PROC DADOS E EMPR.EMP...,21,70.0,1470.0,1176.0,294.0



Relatório de Validação e Ações do Agente:
{'errors': [],
 'fixes': ['Excluídos por cargo: 3 registros',
           "Exclusão por 'ferias': 76 cadastros",
           "Exclusão por 'desligados': 3 cadastros",
           "Exclusão por 'exterior': 1 cadastros",
           "Exclusão por 'afastamentos': 20 cadastros",
           "Exclusão por 'estagio': 3 cadastros"],
 'warnings': []}
